# Report

## Gather

- Neste ponto foram feitos esforços para adquirir os dados necessários para o projeto. Primeiramente, foi gerado um arquivo no formato `TOML` que contém os tokens usados na API do Twitter, este arquivo é aberto como um dicionário pelo python utilizando a extensão toml. As variáveis constantes (em maiúsculo) são criadas com os valores desse token facilitando a manutenção do código e o objeto da API é criado. São utilizados `wait_on_rate_limit` e `wait_on_rate_limit_notify` para que, quando o tempo limite for excedido, a api espere para contínuar e avise dessa pausa;
- Utilizando a url fornecida para o projeto 2, o arquivo `image-predictions.tsv` é carregado como um DataFrame diretamente, aproveitando os recursos das últimas versões do pandas que tem a capacidade de fazer download diretamente ao abrir o url;
- O arquivo `twitter-archive-enhanced.csv` é carregado como o dataframe `twitter_archive`;
- Utilizando os identificadores do dataframe `image_predicion` e a API do twitter, os tweets são baixados no formato json e gravados no arquivo `tweet_json.txt` linha por linha usando a forma de gravação `append` de arquivos do python. Também foi adicionado o comando mágico do ipython `%%time` para calcular o tempo de execução;
- Utilizando o arquivo `tweet_json.txt` e acessar cada linha deste arquivo pegando as informações úteis e adicionando-as a um dataframe vazio chamado `df_tweets`.

## Assess
- Acessa informações úteis de cada dataframe para entender o conteúdo de seus dados, algumas das informações que são acessadas são: `sample(10)` para enxergar 10 amostras aleatórias; `info()` para verificar valores faltantes e os tipos de dados de cada coluna; `duplicated()` para verificar se valores estão duplicados. Ainda foram feitas outras visualizações que ajudaram a esclarecer o que poderia ser melhorado, como retweets existentes e replys de tweets originais;
- Cada link de `extended_urls` foi acessado para descobrir se existiam tweets inexistentes e assim facilitar suas remoções;
- Uma lista de tarefas para melhorar a qualidade e arrumação dos dataframes foi criada para servir como guia na limpeza.

## Clean
- Primeiramente, para cada item da lista foi criada uma definição, ou seja, uma informação do que foi realizado;
- Essa ordem de limpeza foi definida por tabela, onde toda a limpeza de uma tabela era feita antes de prosseguir para a seguinte, a menos que a limpeza não fosse possível;
- A baixo de cada definição, foi realizada a limpeza do código e um ou mais testes para verificar a corretitude da tabela depois da limpeza;
- Ao terminar as limpezas de qualidade se seguiu para as limpezas de arrumação onde foi feita a junção das tabelas com as colunas necessárias e ideais;
- O novo dataframe gerado pela junção ainda teve de receber alguns procedimentos de limpeza que serviram para melhorar a qualidade antes de ser salvo como `twitter_archive_master.csv`

## Analyze
- Algumas pequenas análises descritivas foram realizadas, com a geração de alguns gráficos, e foram escritos alguns insights dos resultados.

# Wrangle Act

## Gather

In [1]:
import re
import json
import toml
import time
import tweepy
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [5]:
# Load twitter tokens
parse_tokens = toml.load('tokens.toml')

# Set tweepy tokens
CONSUMER_KEY = parse_tokens['tokens']['consumer_key']
CONSUMER_SECRET = parse_tokens['tokens']['consumer_secret']
ACCESS_TOKEN = parse_tokens['tokens']['access_token']
ACCESS_SECRET = parse_tokens['tokens']['access_secret']

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [7]:
# Download image_predictions.tsv
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)  
with open('image_predictions.tsv', 'wb') as f:
    f.write(r.content)

image_pretictions = pd.read_csv('image_predictions.tsv', sep='\t')

In [8]:
# Load tweets
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
%%time
# Take tweet json by tweet_id:
# *AVISO*: Execução demorada
id_404_erros = []
# tweets = twitter_archive[(twitter_archive.retweeted_status_id.isnull()) & (twitter_archive.in_reply_to_status_id.isnull())]
tweets = image_pretictions
for tweet_id in tweets.tweet_id:
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        if tweet.user.id == 4196983835:
            with open('tweet_json.txt', 'a') as outfile:
                json.dump(tweet._json, outfile)
                outfile.write('\n')
    except tweepy.TweepError:
        #         if tweepy.TweepError.message[0]['code'] == 404:
        id_404_erros.append(tweet_id)
        continue

In [10]:
# Gera DataFrame de tweets puxados por id
df_tweets = pd.DataFrame(columns=[
                         'tweet_id', 'text', 'retweet_count', 'favorite_count', 'jpg_url', 'url', 'expanded_urls'])

with open('tweet_json.txt', 'r') as file:
    for line in file:
        try:
            line_json = json.loads(line)
            
            if 'media' not in line_json['entities']:
                continue
            line_dict = {
                'tweet_id': line_json['id_str'],
                'text': line_json['full_text'],
                'retweet_count': line_json['retweet_count'],
                'favorite_count': line_json['favorite_count'],
                'jpg_url': line_json['entities']['media'][0]['media_url_https'],
                'url': line_json['entities']['media'][0]['url'],
                'expanded_urls': line_json['entities']['media'][0]['expanded_url']
            }

            df_tweets = df_tweets.append(
                pd.Series(line_dict).to_frame().transpose(), ignore_index=True) # Transpoem do formato de Serie para DataFrame

        except Exception:
            raise Exception(line)
df_tweets.head()

,tweet_id,text,retweet_count,favorite_count,jpg_url,url,expanded_urls
0,666020888022790149,Here we have a Japanese Irish Setter. Lost eye...,504,2533,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,https://t.co/BLDqew2Ijj,https://twitter.com/dog_rates/status/666020888...
1,666029285002620928,This is a western brown Mitsubishi terrier. Up...,47,128,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,https://t.co/r7mOb2m0UI,https://twitter.com/dog_rates/status/666029285...
2,666033412701032449,Here is a very happy pup. Big fan of well-main...,43,124,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,https://t.co/y671yMhoiR,https://twitter.com/dog_rates/status/666033412...
3,666044226329800704,This is a purebred Piers Morgan. Loves to Netf...,139,294,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,https://t.co/DWnyCjf2mx,https://twitter.com/dog_rates/status/666044226...
4,666049248165822465,Here we have a 1949 1st generation vulpix. Enj...,41,107,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,https://t.co/4B7cOc1EDq,https://twitter.com/dog_rates/status/666049248...


## Assess

In [11]:
# Show dataframe loaded head.
image_pretictions.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1954,864197398364647424,https://pbs.twimg.com/media/C_4-8iPV0AA1Twg.jpg,4,golden_retriever,0.945905,True,Labrador_retriever,0.021264,True,Tibetan_mastiff,0.020493,True
915,701214700881756160,https://pbs.twimg.com/media/Cbs3DOAXIAAp3Bd.jpg,1,Chihuahua,0.615163,True,Pembroke,0.159509,True,basenji,0.084466,True
532,676864501615042560,https://pbs.twimg.com/media/CWS0q8iU8AE2Srr.jpg,1,Chesapeake_Bay_retriever,0.371146,True,water_buffalo,0.099596,False,Weimaraner,0.048968,True
628,680889648562991104,https://pbs.twimg.com/media/CXMBhXfWEAA4mMI.jpg,1,Shetland_sheepdog,0.876337,True,collie,0.078331,True,Pomeranian,0.020407,True
16,666102155909144576,https://pbs.twimg.com/media/CT54YGiWUAEZnoK.jpg,1,English_setter,0.298617,True,Newfoundland,0.149842,True,borzoi,0.133649,True
692,684222868335505415,https://pbs.twimg.com/media/CX7Y_ByWwAEJdUy.jpg,1,soft-coated_wheaten_terrier,0.791182,True,cocker_spaniel,0.072444,True,teddy,0.071486,False
544,677301033169788928,https://pbs.twimg.com/media/CWZBsjPWsAAZFFl.jpg,1,Japanese_spaniel,0.661178,True,Pekinese,0.150119,True,Chihuahua,0.119720,True
174,669015743032369152,https://pbs.twimg.com/media/CUjSRNCXAAQ6Y_8.jpg,1,comic_book,0.275927,False,bib,0.173516,False,jersey,0.073911,False
494,675710890956750848,https://pbs.twimg.com/media/CWCbd8ZWoAAtqoH.jpg,2,standard_schnauzer,0.441427,True,miniature_schnauzer,0.248885,True,Sealyham_terrier,0.164967,True
1946,862457590147678208,https://pbs.twimg.com/media/C_gQmaTUMAAPYSS.jpg,1,home_theater,0.496348,False,studio_couch,0.167256,False,barber_chair,0.052625,False


In [12]:
image_pretictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [ ]:
image_pretictions.loc[image_pretictions.jpg_url.isnull(), :]

In [ ]:
twitter_archive.sample(10)

In [ ]:
# Retweets e replys
twitter_archive.info()

In [ ]:
# Formato do texto
twitter_archive.loc[0, 'text']

In [ ]:
# Retweet externo existente
twitter_archive.loc[546, :]

In [ ]:
# Reply
twitter_archive.loc[twitter_archive.in_reply_to_status_id.notnull(), :]
twitter_archive.loc[149, :]

In [ ]:
# Lista de replys
twitter_archive[twitter_archive.in_reply_to_status_id.notnull()]

In [ ]:
# Lista de retweets
twitter_archive[twitter_archive.retweeted_status_id.notnull()]

In [ ]:
# Valores que não foram verificados com machine learning
temp = pd.merge(twitter_archive, image_pretictions,
                on=['tweet_id'], how='outer')
temp[temp.p1.isnull()]

In [ ]:
# Verificando existencia de duplicados
twitter_archive.tweet_id.duplicated().sum()

In [ ]:
image_pretictions.tweet_id.duplicated().sum()

In [ ]:
twitter_archive[twitter_archive.rating_numerator == 1776].text.values

In [ ]:
%%time
# Verificando links de tweets inexistentes
# AVISO: Execução demorada.
count = 0
id_404 = []
for index, row in twitter_archive[twitter_archive.expanded_urls.notna()].iterrows():
    if requests.get(row.expanded_urls.split(',')[0]).status_code == 404:
        print(row.expanded_urls.split(',')[0])
        count += 1
        id_404.append(row.tweet_id)
count

In [ ]:
# Mostra linhas que url retorna erro 404
twitter_archive[twitter_archive.tweet_id.isin(id_404)]

In [ ]:
df_tweets.sample(10)

In [ ]:
df_tweets.info()

In [ ]:
# valores duplicados
df_tweets.tweet_id.value_counts().head(10)

### Quality

#### `image_pretictions` table
- <del>Tipo de dados errado em `tweet_id`;</de>
- <del>Textos com diferentes formas minusculos e maiusculos nas colunas `p1`, `p2` e `p3`;</del>
- <del>Separação dos textos das colunas `p1`, `p2` e `p3` usando `_` ao invés de espaço.</del>

#### `twitter_archive` table
- <del>`in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` com valores incorres no formato `6.67152e+17`;</del>
- <del>Tipo de dados errado em `tweet_id`, `in_reply_to_status_id`, `in_reply_to_user_id`, `timestamp`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`;</del>
- <del>Valores nulos sendo representados como `nan` ao invés de `NaN` nas `colunas in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_id` e `retweeted_status_user_id`;</del>
- <del>Valores nulos sendo representados como `None` em `doggo`, `floofer`, `pupper` e `puppo`;</del>
- <del>Url repetidas na coluna `expanded_urls`;</del>
- <del>Alguns links em expanded_urls levam a páginas com error 404;</de>
- <del>Nem todos os ids tem predições.</del>
- <del>Urls vazias;</del>
- <del>Nomes de pets como `None` e `a`.</del>

#### `df_tweets` table
- <del>Valores nas colunas `retweet_count` e `favorite_count` estão como object ao invés de int.</del>

#### `twitter_archive_master` table
- <del>Limpar urls que não levam a dog_rates</del>

### Tidiness
- <del>Colunas `source` desnecessária em `twitter_archive`;</del>
- <del>Retweets existentes, colunas `retweeted_status_id`, `retweeted_status_user_id` e `retweeted_status_timestamp`.</del>
- <del>Replys existentes, colunas `in_reply_to_status_id`, `in_reply_to_user_id`;</del>
- <del>`text` contém (texto, nota, url) em `twitter_archive` e `df_tweets`;</del>
- <del>Quatro colunas em `twitter_archive` (`doggo`, `floofer`, `pupper` e `puppo`) ao invés de uma;</del>
- <del>Denominador se mostra desnecessário sendo sempre 10 em `twitter_archive` na coluna `rating_denominator`;</del>
- <del>Colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` e `retweeted_status_timestamp`;</del>
- <del>Numero de imagens na coluna `img_num` se mostra desnecessário em `image_predictions`.</del>
- <del>Junção de `df_tweets`, `image_predicitions` e `twitter_archive`;</del>

## Clean

In [ ]:
image_pretictions_clean = image_pretictions.copy()
twitter_archive_clean = twitter_archive.copy()
df_tweets_clean = df_tweets.copy()

### Clean - `image_pretictions` table
#### Define - Tipo de dados errado em `tweet_id`
- Modifica o tipos dos valores da coluna `tweet_id` de inteiro para string usando `astype`

#### Code

In [ ]:
image_pretictions_clean.tweet_id = image_pretictions_clean.tweet_id.astype(
    'str')

#### Test

In [ ]:
image_pretictions_clean.info()

#### Define
- Textos com diferentes formas minusculos e maiusculos nas colunas p1, p2 e p3

#### Code

In [ ]:
image_pretictions_clean.p1 = image_pretictions_clean.p1.apply(
    lambda x: x.lower().replace('_', ' '))
image_pretictions_clean.p2 = image_pretictions_clean.p2.apply(
    lambda x: x.lower().replace('_', ' '))
image_pretictions_clean.p3 = image_pretictions_clean.p3.apply(
    lambda x: x.lower().replace('_', ' '))

#### Test

In [ ]:
image_pretictions_clean.p1.sample(10)

In [ ]:
image_pretictions_clean.p2.sample(10)

In [ ]:
image_pretictions_clean.p3.sample(10)

### Clean - `twitter_archive` table

#### Define - Valores nulos sendo representados como `None` em `doggo`, `floofer`, `pupper` e `puppo`
- Modifica valores `None` para `np.nan` nas colunas `doggo`, `floofer`, `pupper` e `puppo`

#### Code

In [ ]:
twitter_archive_clean.doggo = twitter_archive_clean.doggo.apply(
    lambda x: np.nan if x == 'None' else x)
twitter_archive_clean.floofer = twitter_archive_clean.floofer.apply(
    lambda x: np.nan if x == 'None' else x)
twitter_archive_clean.pupper = twitter_archive_clean.pupper.apply(
    lambda x: np.nan if x == 'None' else x)
twitter_archive_clean.puppo = twitter_archive_clean.puppo.apply(
    lambda x: np.nan if x == 'None' else x)

#### Test

In [ ]:
twitter_archive_clean.doggo.sample(10)

In [ ]:
twitter_archive_clean.floofer.sample(10)

In [ ]:
twitter_archive_clean.pupper.sample(10)

In [ ]:
twitter_archive_clean.puppo.sample(10)

#### Define - Tipo de dados errado em `tweet_id`, `in_reply_to_status_id`, `in_replay_to_user_id`, `timestamp`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`
- Converte int para string nas colunas `tweet_id`, `in_reply_to_status_id`, `in_replay_to_user_id`, `retweeted_status_id` e `retweeted_status_user_id
- Converte string para datetime nas colunas `timestamp` e  `retweeted_status_timestamp`
#### Code

In [ ]:
# Conversão para string
twitter_archive_clean.tweet_id = twitter_archive_clean.tweet_id.astype('str')
twitter_archive_clean.in_reply_to_status_id = twitter_archive_clean.in_reply_to_status_id.astype(
    'str')
twitter_archive_clean.in_reply_to_user_id = twitter_archive_clean.in_reply_to_user_id.astype(
    'str')
twitter_archive_clean.retweeted_status_id = twitter_archive_clean.retweeted_status_id.astype(
    'str')
twitter_archive_clean.retweeted_status_user_id = twitter_archive_clean.retweeted_status_user_id.astype(
    'str')

# Conversão para datatime
twitter_archive_clean.timestamp = pd.to_datetime(
    twitter_archive_clean.timestamp)
twitter_archive_clean.retweeted_status_timestamp = pd.to_datetime(
    twitter_archive_clean.retweeted_status_timestamp)

#### Test

In [ ]:
twitter_archive_clean.info()

In [ ]:
twitter_archive_clean.timestamp.sample(10)

#### Define - Url repetidas na coluna `expanded_urls`
- Linhas onde a url se repete separados por `,` são dividas e apenas o primeiro valor permanece

#### Code

In [ ]:
twitter_archive_clean.expanded_urls = twitter_archive_clean.expanded_urls.apply(
    lambda x: x if x is np.nan else x.split(',')[0])

#### Test

In [ ]:
twitter_archive_clean.expanded_urls.sample(10).values

#### Define - Alguns links em expanded_urls levam a páginas com error 404
- Pela lista criada em `id_404` com identificadores que retornaram erro 404, seleciona as linhas e altera o valor para `np.nan`

#### Code

In [ ]:
twitter_archive_clean.loc[twitter_archive_clean.tweet_id.isin(pd.Series(id_404).astype('str')), 'expanded_urls'] = \
    twitter_archive_clean.loc[twitter_archive_clean.tweet_id.isin(
        pd.Series(id_404).astype('str')), 'expanded_urls'].apply(lambda x: np.nan)

#### Test

In [ ]:
twitter_archive_clean.loc[twitter_archive_clean.tweet_id.isin(
    pd.Series(id_404).astype('str')), 'expanded_urls']

#### Define - Nem todos os ids tem predições
- Remove linhas desnecessárias que não possuem predição, selecionando os ids inexistentes na predição e removendo esses índices.

#### Code

In [ ]:
tweets_no_predictions = twitter_archive_clean[~twitter_archive_clean.tweet_id.isin(
    image_pretictions_clean.tweet_id)]
twitter_archive_clean.drop(tweets_no_predictions.index, axis=0, inplace=True)
# tweets_no_predictions

#### Test

In [ ]:
twitter_archive_clean[~twitter_archive_clean.tweet_id.isin(
    image_pretictions_clean.tweet_id)]

#### Define - Valores nulos sendo representados como `nan` ao invés de `NaN` nas `colunas in_reply_to_user_id`, `in_reply_to_user_id`, `retweeted_status_id` e `retweeted_status_user_id`
- Verifica as linhas em cada coluna e caso encontre `nan` troca por `np.nan`.

#### Code

In [ ]:
twitter_archive_clean.in_reply_to_status_id = twitter_archive_clean.in_reply_to_status_id.apply(
    lambda x: x if x != 'nan' else np.nan)
twitter_archive_clean.in_reply_to_user_id = twitter_archive_clean.in_reply_to_user_id.apply(
    lambda x: x if x != 'nan' else np.nan)
twitter_archive_clean.retweeted_status_id = twitter_archive_clean.retweeted_status_id.apply(
    lambda x: x if x != 'nan' else np.nan)
twitter_archive_clean.retweeted_status_user_id = twitter_archive_clean.retweeted_status_user_id.apply(
    lambda x: x if x != 'nan' else np.nan)

#### Test

In [ ]:
twitter_archive_clean.sample(5)

#### Define - Urls vazias.
- Seleciona as linhas onde a url se encontra como `np.nan` e remove essas linhas pelo id

#### Code

In [ ]:
id_list = twitter_archive_clean[(twitter_archive_clean.expanded_urls.isna())]
twitter_archive_clean.drop(id_list.index, axis=0, inplace=True)

#### Test

In [ ]:
twitter_archive_clean[(twitter_archive_clean.expanded_urls.isna())]

#### Define - Nomes de pets como `None` e `a`.
- Seleciona nomes de pets que se encontram com valores `None` e `a` e os altera para `np.nan`


#### Code

In [ ]:
twitter_archive_clean.name = twitter_archive_clean.name.apply(
    lambda x: np.nan if x == 'None' or x == 'a' else x)

#### Test

In [ ]:
twitter_archive_clean.query('name == "None" or name == "a"')

In [ ]:
twitter_archive_clean.name.sample(10)

### Clean - `df_tweets` table

#### Define - Valores nas colunas retweet_count e favorite_count estão como float ao invés de int.
- Seleciona as colunas `retweet_count` e `favorite_count` e usando astype converte para int.

#### Code

In [ ]:
df_tweets.retweet_count = df_tweets.retweet_count.astype('int')
df_tweets.favorite_count = df_tweets.favorite_count.astype('int')

#### Test

In [ ]:
df_tweets.info()

### Tidiness

#### Define - Colunas `source` desnecessária em `twitter_archive`
- Remove a coluna `source` utilizando `drop`

#### Code

In [ ]:
twitter_archive_clean.drop(columns='source', inplace=True)

#### Test

In [ ]:
twitter_archive_clean.info()

#### Define - Retweets existentes, colunas `retweeted_status_id`, `retweeted_status_user_id` e `retweeted_status_timestamp`
- Seleciona linhas que possuam identificação de reteweets e as remove utilizando `drop`

#### Code

In [ ]:
# Se existe `retweeted_status_id` então existe `retweeted_status_user_id` e `retweeted_status_timestamp`
twitter_archive_clean.drop(
    twitter_archive_clean[twitter_archive_clean.retweeted_status_id.notna()].index, axis=0, inplace=True)

#### Test

In [ ]:
twitter_archive_clean[twitter_archive_clean.retweeted_status_id.notna()]

#### Define - Replys existentes, colunas `in_reply_to_status_id`, `in_reply_to_user_id`.
- Seleciona linhas que possuam identificação de replys e as remove utilizando `drop`


#### Code

In [ ]:
# Se existe `in_reply_to_status_id` então existe `in_reply_to_user_id`
twitter_archive_clean.drop(
    twitter_archive_clean[twitter_archive_clean.in_reply_to_status_id.notna()].index, axis=0, inplace=True)

#### Test

In [ ]:
twitter_archive_clean[twitter_archive_clean.in_reply_to_status_id.notna()]

#### Define - `text` contém (texto, nota, url).
- Limpa a coluna texto removendo a nota e a url utilizando expressão regular, mantendo apenas o texto do tweet

#### Code

In [ ]:
twitter_archive_clean.text = twitter_archive_clean.text.str.replace(r'(\d+/\d+)', '').str.replace(
    r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '').str.strip()
df_tweets_clean.text = df_tweets_clean.text.str.replace(r'(\d+/\d+)', '').str.replace(
    r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '').str.strip()

#### Test

In [ ]:
twitter_archive_clean.text.sample(10).values

In [ ]:
df_tweets_clean.text.sample(10).values

#### Define - Quatro colunas em `twitter_archive` (`doggo`, `floofer`, `pupper` e `puppo`) ao invés de uma.
- Gera uma nova colunas `pet_class` e adiciona os valores `doggo`, `floofer`, `pupper` e `puppo`, ou a junção desses em caso de mais de uma classificação.
- Remove as colunas `doggo`, `floofer`, `pupper` e `puppo` desnecessarias

#### Code

In [13]:
twitter_archive_clean['pet_class'] = twitter_archive_clean[[
    'doggo', 'floofer', 'pupper', 'puppo']].fillna('').sum(axis=1)
twitter_archive_clean.pet_class = twitter_archive_clean.pet_class.apply(
    lambda x: np.nan if x is '' else x)
twitter_archive_clean.drop(
    columns=['doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

NameError: name 'twitter_archive_clean' is not defined

#### Test

In [ ]:
twitter_archive_clean.info()

In [ ]:
twitter_archive_clean.sample(10)

#### Define - Denominador se mostra desnecessário sendo sempre 10 em `twitter_archive` na coluna `rating_denominator` - Colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` e `retweeted_status_timestamp`.
- Remove as colunas `rating_denominator`, `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` e `retweeted_status_timestamp` utilizando `drop`

#### Code

In [ ]:
twitter_archive_clean.drop(columns=['rating_denominator', 'in_reply_to_status_id', 'in_reply_to_user_id',
                                    'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], inplace=True)

#### Test

In [ ]:
twitter_archive_clean.info()

#### Define - Numero de imagens na coluna `img_num` se mostra desnecessário em `image_predictions`.
- Remove a coluna `img_num` utilizando `drop`

#### Code

In [ ]:
image_pretictions_clean.drop(columns='img_num', axis=1, inplace=True)

#### Test

In [ ]:
image_pretictions_clean.info()

#### Define - Junção de df_tweets, image_predicitions e twitter_archive.
- Gera um merge chamado `twitter_archive_master` com a junção das tabelas `twitter_archive` e `df_tweets`
- Junta `twitter_archive_master` com a tabela faltante `image_predicions`

#### Code

In [ ]:
# junta `twitter_archive_clean` com `df_tweets_clean`
twitter_archive_master = twitter_archive_clean.merge(
    df_tweets_clean, how='left', on=['tweet_id', 'text', 'expanded_urls'])

In [ ]:
# junta `twitter_archive_master` com `image_predictions_clean`
twitter_archive_master = twitter_archive_master.merge(
    image_pretictions_clean, how='left', on=['tweet_id', 'jpg_url'])

#### Test

In [ ]:
twitter_archive_master.head()

In [ ]:
twitter_archive_master.info()

### Clean - `twitter_archive_master` table

#### Define - Limpar urls que não levam a dog_rates
- Seleciona urls que não contem `https://twitter.com/dog_rates` e remove as linhas utilizando os índices.
#### Code

In [ ]:
twitter_archive_master_clean = twitter_archive_master.copy()

In [ ]:
# Remove por tweets que não pertençam a https://twitter.com/dog_rates
no_tweets_index = twitter_archive_master_clean[~twitter_archive_master_clean.expanded_urls.str.contains(
    'https://twitter.com/dog_rates', regex=False)].index
twitter_archive_master_clean.drop(no_tweets_index, axis=0, inplace=True)

#### Test

In [ ]:
twitter_archive_master_clean[~twitter_archive_master_clean.expanded_urls.str.contains(
    'https://twitter.com/dog_rates', regex=False)]

### Save clean result

In [ ]:
twitter_archive_master_clean.to_csv('twitter_archive_master.csv', index=False)

## Analyzes

In [ ]:
# carrega o dataframe
df_tweets_archive = pd.read_csv(
    'twitter_archive_master.csv', parse_dates=['timestamp'])
sns.set_style("whitegrid")

In [ ]:
df_tweets_archive.head()

In [ ]:
df_tweets_archive.info()

In [ ]:
df_tweets_archive.describe().to_csv('csv/tab1.csv')
df_tweets_archive.describe()

### Qual a média das notas por classificação?

In [ ]:
df_tweets_archive.rating_numerator.describe().to_csv('csv/tab2.csv')
df_tweets_archive.rating_numerator.describe()

In [ ]:
df_tweets_archive[df_tweets_archive.rating_numerator <=
                  15].rating_numerator.describe().to_csv('csv/tab3.csv')
df_tweets_archive[df_tweets_archive.rating_numerator <=
                  15].rating_numerator.describe()

In [ ]:
ax = sns.boxplot(
    data=df_tweets_archive[df_tweets_archive.rating_numerator <= 15].rating_numerator)
ax.set_title('Boxplot das notas atribuídas aos animais')
ax.set_ylabel('Notas')
ax.set_xticks([])
fig = ax.get_figure()
fig.savefig('imgs/fig1.png');

#### Insight

Neste gráfico podemos verificar depois de remover as notas discrepantes, ou seja superiores a 15, assim removendo outliers (valores que podiam chegar a 1776, sendo que a nota deveria ser até 10 visto que o denominador é 10 mas os usuários como brincadeira classificam os animais com notas a cima desse valor), e com média de 10.49 e desvio de 2.19. Observa-se que existe uma homogeniedade da variância no boxplot, visto que a diferença entre o terceiro quartil e a mediana assim como a diferença entre o primeiro quartil e a mediana parecem ser a mesmas.

### Relação entre contagem de retweets e tweets favoritos

In [ ]:
sns.set_style("whitegrid")

ax = sns.scatterplot(x="retweet_count", y="favorite_count",
                     hue='pet_class', data=df_tweets_archive)
ax.set_title('Contagens de retweets por tweets favoritados por classificação')
ax.set_xlabel('Contagem retweets')
ax.set_ylabel('Contagem tweets favoritos')
fig = ax.get_figure()
fig.savefig('imgs/fig2.png');

#### Insight

Esse é um gráfico de dispersão, entre a contagem de retweets e a contagem de tweets favoritos para cada classificação. Observa-se que a medida que aumenta a contagem de retweets aumenta a contagem de tweets favoritos (o que pode ser devido a maior visibilidade do tweet) e a classe que possui a maior evidência é a classe pupper. Vale salientar que é possível observar que a classe doggo possui observações com maiores números de retweets, consequentemente, maior contagem de tweets favoritos.

In [ ]:
ax = sns.scatterplot(x="pet_class", y="retweet_count", data=df_tweets_archive)
labels = df_tweets_archive.pet_class.unique()[1:]
ax.set_xticklabels(rotation=45, ha='right', labels=labels)
ax.set_title('Contagens de retweets por classificação')
ax.set_xlabel('Classificação dos animais')
ax.set_ylabel('Contagem de retweets')
fig = ax.get_figure()
fig.savefig('imgs/fig3.png');

#### Insight

Por este gráfico é possível verificar que realmente a classificação doggo é a que possui o tweet com maior número de retweets, porém também podemos ver que pupper tem um grande concentração de retweets até 20000. Cachorros que possuem duas classificações são poucos com relação aos demais.

In [ ]:
ax = sns.scatterplot(x="pet_class", y="favorite_count", data=df_tweets_archive)
labels = df_tweets_archive.pet_class.unique()[1:]
ax.set_xticklabels(rotation=45, ha='right', labels=labels)
ax.set_title('Contagens de tweets favoritados por classificação')
ax.set_xlabel('Classificação dos animais')
ax.set_ylabel('Contagem de tweets favoritos')
fig = ax.get_figure()
fig.savefig('imgs/fig4.png');

In [ ]:
df_tweets_archive[['retweet_count', 'favorite_count']].corr().to_csv('csv/tab4.csv')
df_tweets_archive[['retweet_count', 'favorite_count']].corr()

#### Insight

Ao comparar a correlação entre `retweet_count` e `favorite_count` pode-se notar que existe uma forte correlação positiva entre as variáveis, isto é a médida que a contagem de retweets aumenta significa o quão favorito aquele tweet é com os usuários.

### Contagem de tweets por data da post e classificação do animal

In [ ]:
graph = df_tweets_archive.set_index('timestamp')
graph = graph.groupby([pd.Grouper(freq='M'), 'pet_class']).count()
graph = graph.reset_index(level=['timestamp', 'pet_class'])

ax = sns.lineplot(x='timestamp', y='tweet_id', data=graph, hue='pet_class')
ax.set_xticklabels(rotation=45, ha='right', labels=graph.timestamp.dt.date)
ax.set_title('Contagem de tweets por data e classificação')
ax.set_xlabel('Data')
ax.set_ylabel('Contagem')
fig = ax.get_figure()
fig.savefig('imgs/fig5.png');

#### Insight

Ao inicio é possível observar que a grande maioria dos tweets com classificação eram classificados unicamente como pupper, a medida que o tempo passa outras classificações são utilizadas e a contagem de tweets de cachorros denominados como pupper diminuiu. Observa-se que embora tenham ocorrido diferentes classificações, como doggo por exemplo, o número de tweets que possuem os animais denominados nessas classes diminuiu entre os anos.